In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn
import torch.optim as optim
import torch.cuda.amp as amp
from torch.cuda.amp import GradScaler
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, models
import torchvision.transforms as transforms
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Enhanced Configuration
class Config:
    # Paths
    DATA_PATH = '/kaggle/input/sheep-classification-challenge-2025/Sheep Classification Images'
    TRAIN_DIR = os.path.join(DATA_PATH, 'train')
    TEST_DIR = os.path.join(DATA_PATH, 'test')
    TRAIN_CSV = os.path.join(DATA_PATH, 'train_labels.csv')
    
    # Enhanced Model parameters
    IMG_SIZE = 456  # Increased from 380 - better resolution helps
    BATCH_SIZE = 6  # Reduced to allow larger images
    NUM_EPOCHS = 100  # Increased epochs
    LEARNING_RATE = 2e-5  # Lower learning rate for fine-tuning
    NUM_CLASSES = 7
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Enhanced Early stopping
    EARLY_STOPPING_PATIENCE = 15  # Increased patience
    MIN_DELTA = 0.0005  # More sensitive improvement detection
    
    # Breeds
    BREEDS = ['Naeimi', 'Najdi', 'Harri', 'Goat', 'Sawakni', 'Roman', 'Barbari']
    
    # Enhanced model architecture
    MODEL_NAME = 'efficientnet_b5'  # Upgraded from B4 to B5
    
    # Cross-validation folds
    N_FOLDS = 5

config = Config()

print(f"Using device: {config.DEVICE}")
print(f"Enhanced configuration: {config.IMG_SIZE}px, {config.MODEL_NAME}")

# Enhanced Data Augmentation
def get_enhanced_transforms():
    """Enhanced data augmentation for better generalization"""
    train_transform = transforms.Compose([
        transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
        # Transforms that operate on PIL Images go first
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(degrees=20),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.15),
        transforms.RandomAffine(degrees=0, translate=(0.15, 0.15), scale=(0.85, 1.15)),
        transforms.RandomPerspective(distortion_scale=0.2, p=0.3),
        
        # Convert to Tensor BEFORE operations that require tensors (like RandomErasing)
        transforms.ToTensor(),
        
        # Transforms that operate on Tensors go after ToTensor()
        transforms.RandomErasing(p=0.2, scale=(0.02, 0.1)), # This now operates on a tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform
    
# def get_enhanced_transforms():
#     """Enhanced data augmentation for better generalization"""
#     train_transform = transforms.Compose([
#         transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
#         # More aggressive augmentations
#         transforms.RandomHorizontalFlip(p=0.5),
#         transforms.RandomVerticalFlip(p=0.2),  # New
#         transforms.RandomRotation(degrees=20),  # Increased
#         transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.15),  # Enhanced
#         transforms.RandomAffine(degrees=0, translate=(0.15, 0.15), scale=(0.85, 1.15)),  # Enhanced
#         transforms.RandomPerspective(distortion_scale=0.2, p=0.3),  # New
#         transforms.RandomErasing(p=0.2, scale=(0.02, 0.1)),  # New - helps with overfitting
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#     ])
    
#     val_transform = transforms.Compose([
#         transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#     ])
    
#     return train_transform, val_transform

def get_enhanced_tta_transforms():
    """Enhanced TTA with more variations"""
    tta_transforms = [
        # Original
        transforms.Compose([
            transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        # Horizontal flip
        transforms.Compose([
            transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
            transforms.RandomHorizontalFlip(p=1.0),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        # Rotation variants
        transforms.Compose([
            transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
            transforms.RandomRotation(degrees=(5, 5)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        transforms.Compose([
            transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
            transforms.RandomRotation(degrees=(-5, -5)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        # Brightness variants
        transforms.Compose([
            transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
            transforms.ColorJitter(brightness=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        # Scale variants
        transforms.Compose([
            transforms.Resize((int(config.IMG_SIZE * 1.05), int(config.IMG_SIZE * 1.05))),
            transforms.CenterCrop(config.IMG_SIZE),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    ]
    return tta_transforms


class EnhancedSheepClassifier(nn.Module):
    def __init__(self, num_classes=7, model_name='efficientnet_b5'):
        super(EnhancedSheepClassifier, self).__init__()
        
        if model_name == 'efficientnet_b5':
            self.backbone = models.efficientnet_b5(weights=models.EfficientNet_B5_Weights.IMAGENET1K_V1)
            # Get the number of features right before the final classification layer
            in_features = self.backbone.classifier[1].in_features
            
            # Replace the classifier with a more robust head.
            # The backbone already handles AdaptiveAvgPool2d and Flatten internally.
            # So, we directly use the in_features for our custom layers.
            self.backbone.classifier = nn.Sequential(
                nn.BatchNorm1d(in_features), # Input here should be (batch_size, in_features)
                nn.Dropout(0.4),
                nn.Linear(in_features, 1024),
                nn.BatchNorm1d(1024),
                nn.ReLU(inplace=True),
                nn.Dropout(0.4),
                nn.Linear(1024, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(inplace=True),
                nn.Dropout(0.3),
                nn.Linear(512, num_classes)
            )
        elif model_name == 'efficientnet_b4':
            self.backbone = models.efficientnet_b4(weights=models.EfficientNet_B4_Weights.IMAGENET1K_V1)
            in_features = self.backbone.classifier[1].in_features
            self.backbone.classifier = nn.Sequential(
                nn.BatchNorm1d(in_features),
                nn.Dropout(0.4),
                nn.Linear(in_features, 768),
                nn.BatchNorm1d(768),
                nn.ReLU(inplace=True),
                nn.Dropout(0.3),
                nn.Linear(768, 384),
                nn.BatchNorm1d(384),
                nn.ReLU(inplace=True),
                nn.Dropout(0.2),
                nn.Linear(384, num_classes)
            )

    def forward(self, x):
        return self.backbone(x)

# Enhanced Dataset Class
class EnhancedSheepDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, is_test=False):
        self.df = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        
        if not is_test:
            self.label_encoder = LabelEncoder()
            self.labels = self.label_encoder.fit_transform(self.df['label'])
            self.label_to_idx = {label: idx for idx, label in enumerate(self.label_encoder.classes_)}
            self.idx_to_label = {idx: label for label, idx in self.label_to_idx.items()}
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        filename = self.df.iloc[idx]['filename']
        img_path = os.path.join(self.img_dir, filename)
        
        try:
            image = Image.open(img_path).convert('RGB')
            # Enhanced image preprocessing
            image = self.enhance_image_quality(image)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            image = Image.new('RGB', (config.IMG_SIZE, config.IMG_SIZE), (0, 0, 0))
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:
            return image, filename
        else:
            label = self.labels[idx]
            return image, label
    
    def enhance_image_quality(self, image):
        """Apply basic image enhancement"""
        # Convert to numpy for processing
        img_array = np.array(image)
        
        # Slight contrast enhancement
        img_array = np.clip(img_array * 1.05, 0, 255).astype(np.uint8)
        
        return Image.fromarray(img_array)

# Cross-Validation Training
def cross_validate_model(train_df):
    """Perform stratified k-fold cross-validation"""
    skf = StratifiedKFold(n_splits=config.N_FOLDS, shuffle=True, random_state=42)
    fold_scores = []
    fold_models = []
    
    train_transform, val_transform = get_enhanced_transforms()
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['label'])):
        print(f"\n{'='*20} FOLD {fold+1}/{config.N_FOLDS} {'='*20}")
        
        fold_train_df = train_df.iloc[train_idx].reset_index(drop=True)
        fold_val_df = train_df.iloc[val_idx].reset_index(drop=True)
        
        # Create datasets
        train_dataset = EnhancedSheepDataset(fold_train_df, config.TRAIN_DIR, train_transform)
        val_dataset = EnhancedSheepDataset(fold_val_df, config.TRAIN_DIR, val_transform)
        
        train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=2)
        val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2)
        
        # Initialize model
        model = EnhancedSheepClassifier(num_classes=config.NUM_CLASSES, model_name=config.MODEL_NAME)
        model = model.to(config.DEVICE)
        
        # Calculate class weights
        train_labels_numerical = train_dataset.labels
        class_weights_array = compute_class_weight(
            class_weight='balanced',
            classes=np.unique(train_labels_numerical),
            y=train_labels_numerical
        )
        if config.DEVICE.type == 'cuda':
            class_weights_tensor = torch.tensor(class_weights_array, dtype=torch.float16).to(config.DEVICE)
        else:
            class_weights_tensor = torch.tensor(class_weights_array, dtype=torch.float32).to(config.DEVICE)
        # class_weights_tensor = torch.tensor(class_weights_array, dtype=torch.float).to(config.DEVICE)
        
        # Enhanced optimizer and scheduler
        criterion = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=0.1)  # Label smoothing
        optimizer = optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=2e-4)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
        
        # Train fold
        best_model, best_f1 = train_single_fold(model, train_loader, val_loader, criterion, optimizer, scheduler, fold)
        
        fold_scores.append(best_f1)
        fold_models.append(best_model)
        
        print(f"Fold {fold+1} Best F1: {best_f1:.4f}")
    
    print(f"\nCross-Validation Results:")
    print(f"Mean F1: {np.mean(fold_scores):.4f} ± {np.std(fold_scores):.4f}")
    print(f"Individual Fold Scores: {[f'{score:.4f}' for score in fold_scores]}")
    
    return fold_models, fold_scores

def train_single_fold(model, train_loader, val_loader, criterion, optimizer, scheduler, fold):
    """Train a single fold"""
    best_f1 = 0.0
    best_model_state = None
    patience_counter = 0
    scaler = GradScaler()
    
    for epoch in range(config.NUM_EPOCHS):
        # Training phase
        model.train()
        running_loss = 0.0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(config.DEVICE), target.to(config.DEVICE)
            
            optimizer.zero_grad()
            with amp.autocast():
                output = model(data)
                loss = criterion(output, target)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(config.DEVICE), target.to(config.DEVICE)
                with amp.autocast():
                    output = model(data)
                val_loss += criterion(output, target).item()
                
                _, predicted = torch.max(output.data, 1)
                all_preds.extend(predicted.cpu().numpy())
                all_targets.extend(target.cpu().numpy())
        
        f1 = f1_score(all_targets, all_preds, average='macro')
        
        if f1 > best_f1:
            best_f1 = f1
            best_model_state = model.state_dict().copy()
            patience_counter = 0
            print(f"Fold {fold+1}, Epoch {epoch+1}: New best F1 = {best_f1:.4f}")
        else:
            patience_counter += 1
        
        scheduler.step()
        
        if patience_counter >= config.EARLY_STOPPING_PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    # Load best model
    model.load_state_dict(best_model_state)
    return model, best_f1

# Enhanced Ensemble Prediction
def ensemble_predict(models, test_df, label_encoder):
    """Make ensemble predictions using multiple models and TTA"""
    tta_transforms = get_enhanced_tta_transforms()
    all_predictions = []
    
    for model_idx, model in enumerate(models):
        model.eval()
        model_predictions = []
        
        print(f"Generating predictions for model {model_idx+1}/{len(models)}...")
        
        for idx in tqdm(range(len(test_df)), desc=f"Model {model_idx+1}"):
            filename = test_df.iloc[idx]['filename']
            img_path = os.path.join(config.TEST_DIR, filename)
            
            try:
                image = Image.open(img_path).convert('RGB')
                # Apply same enhancement as training
                dataset = EnhancedSheepDataset(pd.DataFrame(), '', None, True)
                image = dataset.enhance_image_quality(image)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
                image = Image.new('RGB', (config.IMG_SIZE, config.IMG_SIZE), (0, 0, 0))
            
            tta_outputs = []
            with torch.no_grad():
                for tta_transform in tta_transforms:
                    augmented_image = tta_transform(image).unsqueeze(0).to(config.DEVICE)
                    with amp.autocast():
                        output = model(augmented_image)
                    tta_outputs.append(torch.softmax(output.float(), dim=1))
            
            # Average TTA predictions
            avg_output = torch.mean(torch.cat(tta_outputs, dim=0), dim=0)
            model_predictions.append(avg_output.cpu().numpy())
        
        all_predictions.append(np.array(model_predictions))
    
    # Ensemble averaging
    ensemble_probs = np.mean(all_predictions, axis=0)
    final_predictions = np.argmax(ensemble_probs, axis=1)
    
    # Convert to labels
    pred_labels = label_encoder.inverse_transform(final_predictions)
    
    return test_df['filename'].values, pred_labels

# Enhanced main function
def enhanced_main():
    """Enhanced main pipeline with cross-validation and ensemble"""
    print("=== Enhanced Sheep Classification Pipeline ===")
    print(f"Target: Top 10 (>0.98 F1)")
    print(f"Enhanced Model: {config.MODEL_NAME} with {config.IMG_SIZE}px resolution")
    print("=" * 60)
    
    # Load data
    train_df = pd.read_csv(config.TRAIN_CSV)
    print(f"Training data: {train_df.shape}")
    print(f"Class distribution:\n{train_df['label'].value_counts()}")
    
    # Cross-validation training
    print(f"\nStarting {config.N_FOLDS}-fold cross-validation...")
    fold_models, fold_scores = cross_validate_model(train_df)
    
    # Prepare test data
    test_files = [f for f in os.listdir(config.TEST_DIR) if f.endswith('.jpg')]
    test_df = pd.DataFrame({'filename': test_files})
    
    # Get label encoder from first model (they should all be the same)
    temp_dataset = EnhancedSheepDataset(train_df, config.TRAIN_DIR, None, is_test=False)
    label_encoder = temp_dataset.label_encoder
    
    # Ensemble prediction
    print(f"\nGenerating ensemble predictions on {len(test_df)} test images...")
    filenames, predictions = ensemble_predict(fold_models, test_df, label_encoder)
    
    # Create submission
    submission_df = pd.DataFrame({
        'filename': filenames,
        'label': predictions
    })
    
    submission_df.to_csv('enhanced_submission.csv', index=False)
    
    print(f"\nEnhanced Results:")
    print(f"Cross-validation mean F1: {np.mean(fold_scores):.4f} ± {np.std(fold_scores):.4f}")
    print(f"Ensemble submission created: enhanced_submission.csv")
    print(f"Test predictions: {len(predictions)}")
    print(f"\nPrediction distribution:")
    print(submission_df['label'].value_counts())
    
    return fold_models, submission_df, fold_scores

# Run enhanced pipeline
if __name__ == "__main__":
    models, submission, scores = enhanced_main()

Using device: cuda
Enhanced configuration: 456px, efficientnet_b5
=== Enhanced Sheep Classification Pipeline ===
Target: Top 10 (>0.98 F1)
Enhanced Model: efficientnet_b5 with 456px resolution
Training data: (682, 2)
Class distribution:
label
Naeimi     255
Goat       107
Sawakni     80
Roman       72
Najdi       71
Harri       62
Barbari     35
Name: count, dtype: int64

Starting 5-fold cross-validation...

==================== FOLD 1/5 ====================
Fold 1, Epoch 1: New best F1 = 0.2347
Fold 1, Epoch 3: New best F1 = 0.3283
Fold 1, Epoch 4: New best F1 = 0.4320
Fold 1, Epoch 5: New best F1 = 0.4663
Fold 1, Epoch 6: New best F1 = 0.4675
Fold 1, Epoch 7: New best F1 = 0.5196
Fold 1, Epoch 10: New best F1 = 0.5418
Fold 1, Epoch 11: New best F1 = 0.6100
